In [1]:
import requests
import dotenv
import os, time
import pandas as pd

dotenv.load_dotenv()

True

In [2]:
scratch = os.getenv('SCRATCH')
hashtag_folder = f"{scratch}/tiktok/hashtag_data"
oak = os.getenv('OAK')
# video_folder = f"{oak}/samori/tiktok/video_data_known_keywords"
video_folder = f"{oak}/samori/tiktok/test_folder2"
os.path.exists(video_folder)

True

In [3]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        
        return response_json
        
    else:
        # If the request was not successful, print the error response JSON
        print("Error:", response.json())

In [4]:
client_key = os.getenv("CLIENT_KEY")
client_secret = os.getenv("CLIENT_SECRET")

credentials = get_access_token(client_key, client_secret)
access_token = credentials['access_token']
token_type = credentials['token_type']

In [5]:
formal_terms = ["fentanyl", "opioids", "opiates", "morphine", "codeine", "oxycodone", "oxymorphone", "mscontin", "percocet"]
informal_terms = ["sublimaze", "duragesic", "fentanil", "sufentanil", "fentanylum", "fentora", "thebaine", "codiene", "roxanol", 
                     "kadian", "oxycontin", "roxicodone", "roxicet", "endocet", "endocodone", "oxyir", "oxynorm", "hydrocodone", 
                     "vicodinsublimaze", "duragesic", "fentanil", "sufentanil", "fentanylum", "fentora", "thebaine", "codiene", 
                     "roxanol", "kadian", "oxycontin", "roxicodone", "roxicet", "endocet", "endocodone", "oxyir", "oxynorm", 
                     "hydrocodone", "vicodin"]

algospeak_terms = ["paink!ller", "f3nt@nol", "cod3in3", "c0d0n3", "f3nt4nol", "p@1nk!ller", "oxy80s", "0xy80", "m0rph!n3", "m3rph0n3", 
                   "m0rf3n", "c0d3in", "0xyc", "s!zzurp", "0pana", "0xym0rph0ne", "num0rph@n", "m0rf33n"]


In [6]:
fields = "id,video_description,create_time,region_code,share_count,view_count,like_count,comment_count,music_id,hashtag_names,username,effect_ids,playlist_id,voice_to_text,is_stem_verified,favorites_count,video_duration"
normal_keywords = formal_terms + informal_terms

In [36]:
start_date = "20220102"
end_date = "20220115" 

endpoint = "https://open.tiktokapis.com/v2/research/video/query/"

query_params = {
    "fields": fields
}

query_body = {
    "query": {
        "and": [
        {
            "operation":"IN",
            "field_name":"keyword",
            "field_values":["animal"]
        },
        {
            "operation":"EQ",
            "field_name":"region_code",
            "field_values":["US"]
        }
        ]
    },
    "max_count": 100,
    "start_date": start_date,
    "end_date": end_date
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
    }

In [37]:
response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)
print(response.status_code)

500


In [38]:
response.json()

{'error': {'code': 'internal_error',
  'message': 'Server Internal Error',
  'log_id': '202409030249109892239942FB3A9B0194'}}

In [17]:
data = response.json().get("data", {})

search_id = data["search_id"]
has_more = data["has_more"]
cursor = data["cursor"]

videos = data.get("videos", [])
df = pd.DataFrame(videos)
df.to_csv(f"{hashtag_folder}/hashtags_{start_date}_{end_date}_{cursor}_{search_id}.csv", index=False)

print("Search ID: ", search_id)
print(cursor, has_more, len(df))

Search ID:  7409072743070700586
100 True 86


In [8]:
# details below for jun 01 - jun 15
# search_id = "7408038137778181162"
# has_more = True
# cursor = 32500
# # hashtags_20240601_20240615_9900_7408038137778181162
# # 32500

In [74]:
search_id = "7409162096480654378"
has_more = True
cursor = 6120

In [62]:
cursor

5800

In [75]:
while has_more:
    
    query_body.update({
        "search_id": search_id,
        "cursor": cursor})
    
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code != 200:
        i = 0
        while i < 5 and response.status_code != 200:
            response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)
            time.sleep(5)
            i += 1
        if response.status_code != 200:
            print(response.status_code)
            print(response.json())
            break
            
    data = response.json().get("data", {})

    search_id = data["search_id"]
    has_more = data["has_more"]
    cursor = data["cursor"]

    videos = data.get("videos", [])
    df = pd.DataFrame(videos)
    
    # df.to_csv(f"./hashtag_data/hashtags_{start_date}_{end_date}_{cursor}.csv", index=False)
    # df.to_csv(f"{hashtag_folder}/hashtags_{start_date}_{end_date}_{cursor}_{search_id}.csv", index=False)
    df.to_csv(f"{video_folder}/videos_{start_date}_{end_date}_{cursor}_{search_id}.csv", index=False)
    
    
    time.sleep(5)
    print(cursor, has_more, len(df), search_id)

    # if cursor > 800:
    #     break

6220 True 60 7409162096480654378
6320 True 77 7409162096480654378
6420 True 65 7409162096480654378
6520 True 71 7409162096480654378
6524 False 3 7409162096480654378


In [19]:
cursor

700

In [28]:
response.json()

{'error': {'code': 'invalid_params',
  'message': 'Search Id 7408027535520551979 is invalid or expired',
  'log_id': '20240828033812355512ACD27525003BC7'}}

In [4]:
query_body.update({
        "search_id": "a",
        "cursor": "c"
    })